In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')


Since we were provided with 2 datasets, with train and test datasets, so we are importing both the datasets together in 1 cell

In [2]:
data=pd.read_csv('fraudTrain.csv') #training dataset
data_test=pd.read_csv('fraudTest.csv') #testing dataset

In [3]:
data.head()

Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652095   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ...      lat      long  \
0    Banks      F                561 Perry Cove  ...  36.0788  -81.1781   
1     Gill      F  43039 Riley Greens Suite 393  ...  48.8878 -118.2105   
2  Sanchez      M      594 White Dale Suite 530  ...  42.1808 -112.2620   
3    White      M   9443 Cynthia Court Apt. 038  ...  46.2306 -112.1138   
4   Garcia      M              408 Bradley Rest  ...  38.4207  -79.4629   

   city_pop                                job         dob  \
0      3495          Psychologist, counselling  1988-03-09   
1       149  Special educational needs teacher  1978-06-21   
2      4154        Nature conservation officer  1962-01-19   
3      1939                    Patent attorney  1967-01-12   
4        99     Dance movement psychotherapist  1986-03-28   

                          trans_num   unix_time  merch_lat  merch_long  \
0  0b242abb623afc578575680df30655b9  1325376018  36.011293  -82.048315   
1  1f76529f8574734946361c461b024d99  1325376044  49.159047 -118.186462   
2  a1a22d70485983eac12b5b88dad1cf95  1325376051  43.150704 -112.154481   
3  6b849c168bdad6f867558c3793159a81  1325376076  47.034331 -112.561071   
4  a41d7549acf90789359a9aa5346dcb46  1325376186  38.674999  -78.632459   

   is_fraud  
0         0  
1         0  
2         0  
3         0  
4         0  

[5 rows x 23 columns]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [5]:
#printing no of unique values
for i in (data.columns):
    print(i,"________",len(pd.unique(data[i])))

Unnamed: 0 ________ 1296675
trans_date_trans_time ________ 1274791
cc_num ________ 983
merchant ________ 693
category ________ 14
amt ________ 52928
first ________ 352
last ________ 481
gender ________ 2
street ________ 983
city ________ 894
state ________ 51
zip ________ 970
lat ________ 968
long ________ 969
city_pop ________ 879
job ________ 494
dob ________ 968
trans_num ________ 1296675
unix_time ________ 1274823
merch_lat ________ 1247805
merch_long ________ 1275745
is_fraud ________ 2


In [6]:
data.drop(['Unnamed: 0','first','last','trans_num','street','city'],axis=1,inplace=True)

In [7]:
data.head()

trans_date_trans_time            cc_num                            merchant  \
0   2019-01-01 00:00:18  2703186189652095          fraud_Rippin, Kub and Mann   
1   2019-01-01 00:00:44      630423337322     fraud_Heller, Gutmann and Zieme   
2   2019-01-01 00:00:51    38859492057661                fraud_Lind-Buckridge   
3   2019-01-01 00:01:16  3534093764340240  fraud_Kutch, Hermiston and Farrell   
4   2019-01-01 00:03:06   375534208663984                 fraud_Keeling-Crist   

        category     amt gender state    zip      lat      long  city_pop  \
0       misc_net    4.97      F    NC  28654  36.0788  -81.1781      3495   
1    grocery_pos  107.23      F    WA  99160  48.8878 -118.2105       149   
2  entertainment  220.11      M    ID  83252  42.1808 -112.2620      4154   
3  gas_transport   45.00      M    MT  59632  46.2306 -112.1138      1939   
4       misc_pos   41.96      M    VA  24433  38.4207  -79.4629        99   

                                 job         dob   unix_time  merch_lat  \
0          Psychologist, counselling  1988-03-09  1325376018  36.011293   
1  Special educational needs teacher  1978-06-21  1325376044  49.159047   
2        Nature conservation officer  1962-01-19  1325376051  43.150704   
3                    Patent attorney  1967-01-12  1325376076  47.034331   
4     Dance movement psychotherapist  1986-03-28  1325376186  38.674999   

   merch_long  is_fraud  
0  -82.048315         0  
1 -118.186462         0  
2 -112.154481         0  
3 -112.561071         0  
4  -78.632459         0

In [8]:
data["recency"] = data.groupby(by="cc_num")["unix_time"].diff()
data["recency"].isnull().sum()

983

In [9]:
# we are getting null values because as 983 because there are 983 unique values of cards, this means whenever the cc_num group changes
# python makes the first value of every group null, so making them as starting payment, we will initialize null values to -1

data.loc[data.recency.isnull(),["recency"]] = -1

In [10]:
# converting trans_date_trans_time to datetime

data["trans_date_trans_time"] = pd.to_datetime(data["trans_date_trans_time"])

In [11]:
# we now don't need unix_time so we can drop it
data= data.drop("unix_time",1)

In [12]:
#as dob is not nessery but year old would be a usefull infomation
#so converting it to year old according to year 2022
data['year_old']=data['dob'].str.split('-').apply(lambda x,y=2022:y-int(x[0]))

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   trans_date_trans_time  1296675 non-null  datetime64[ns]
 1   cc_num                 1296675 non-null  int64         
 2   merchant               1296675 non-null  object        
 3   category               1296675 non-null  object        
 4   amt                    1296675 non-null  float64       
 5   gender                 1296675 non-null  object        
 6   state                  1296675 non-null  object        
 7   zip                    1296675 non-null  int64         
 8   lat                    1296675 non-null  float64       
 9   long                   1296675 non-null  float64       
 10  city_pop               1296675 non-null  int64         
 11  job                    1296675 non-null  object        
 12  dob                    12966

In [14]:
# as now dob is no use
data.drop('dob',axis=1,inplace=True)

In [15]:
# sometimes distance from the customer's home location to the merchant's location can prove out to be main reason for fraud,
#so taking the difference of longitude and lattitude of respective columns

data["lat_diff"] = abs(data.lat - data.merch_lat)
data["long_diff"] = abs(data["long"] - data["merch_long"])

# we have used abs function so that we get proper distance diiference in positive as abs makes negative values positive
#and used as a mod function


# now since we have the difference, it is estimated that difference between each degree of longitude and lattitude is 69 miles(approx)
# or 110 kilometers (approx), so taking displacement into account as it will be difficult to calculate distance between merchant's location
# or customer's location so applying pythogoras theorem

data["displacement"] = np.sqrt(pow((data["lat_diff"]*110),2) + pow((data["long_diff"]*110),2))

# here we have applied pythogoras theorem and we have multiplied with 110 because each degree of longitude and lattitude is 69 miles(approx)
# or 110 kilometers apart


In [16]:
# now since we got the displacement so longitudes and lattitudes columns are of no use now, so we can remove them
data.drop(columns = ["lat","long","merch_lat","merch_long","lat_diff","long_diff"],axis=1,inplace=True)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   trans_date_trans_time  1296675 non-null  datetime64[ns]
 1   cc_num                 1296675 non-null  int64         
 2   merchant               1296675 non-null  object        
 3   category               1296675 non-null  object        
 4   amt                    1296675 non-null  float64       
 5   gender                 1296675 non-null  object        
 6   state                  1296675 non-null  object        
 7   zip                    1296675 non-null  int64         
 8   city_pop               1296675 non-null  int64         
 9   job                    1296675 non-null  object        
 10  is_fraud               1296675 non-null  int64         
 11  recency                1296675 non-null  float64       
 12  year_old               12966

In [18]:
data.drop('zip',axis=1,inplace=True) 

In [19]:
# Although date part in column trans_date_trans_time is not important because that is historical data, but time part of the component is important
# so creating a column of time

data["Time"] = pd.to_datetime(data["trans_date_trans_time"],"%H:%M").dt.time


In [20]:
# converting Time column to datetime
data["Time"] = pd.to_datetime(data["trans_date_trans_time"]).dt.hour

In [21]:
# recency column converting them from seconds to minutes
data.recency = data.recency.apply(lambda x: float((x/60)/60))

# now working on test data

In [22]:
#droping all unnessery data
data_test.drop(['Unnamed: 0','first','last','trans_num','street','city','zip'],axis=1,inplace=True)

In [23]:
data_test["recency"] = data_test.groupby(by="cc_num")["unix_time"].diff()
data_test.loc[data_test.recency.isnull(),["recency"]] = -1

In [24]:
data_test["trans_date_trans_time"] = pd.to_datetime(data_test["trans_date_trans_time"])

In [25]:
data_test['year_old']=data_test['dob'].str.split('-').apply(lambda x,y=2022:y-int(x[0]))

In [26]:
data_test["lat_diff"] = abs(data_test.lat - data_test.merch_lat)
data_test["long_diff"] = abs(data_test["long"] - data_test["merch_long"])
data_test["displacement"] = np.sqrt(pow((data_test["lat_diff"]*110),2) + pow((data_test["long_diff"]*110),2))

In [27]:
data_test.recency = data_test.recency.apply(lambda x: float((x/60)/60))

In [28]:
data_test["Time"] = pd.to_datetime(data_test["trans_date_trans_time"],"%H:%M").dt.time
data_test["Time"] = pd.to_datetime(data_test["trans_date_trans_time"]).dt.hour

In [29]:
data_test.drop(['lat', 'long', 'dob', 'unix_time','merch_lat', 'merch_long','lat_diff', 'long_diff'],axis=1,inplace=True)

In [30]:
data.head()

trans_date_trans_time            cc_num                            merchant  \
0   2019-01-01 00:00:18  2703186189652095          fraud_Rippin, Kub and Mann   
1   2019-01-01 00:00:44      630423337322     fraud_Heller, Gutmann and Zieme   
2   2019-01-01 00:00:51    38859492057661                fraud_Lind-Buckridge   
3   2019-01-01 00:01:16  3534093764340240  fraud_Kutch, Hermiston and Farrell   
4   2019-01-01 00:03:06   375534208663984                 fraud_Keeling-Crist   

        category     amt gender state  city_pop  \
0       misc_net    4.97      F    NC      3495   
1    grocery_pos  107.23      F    WA       149   
2  entertainment  220.11      M    ID      4154   
3  gas_transport   45.00      M    MT      1939   
4       misc_pos   41.96      M    VA        99   

                                 job  is_fraud   recency  year_old  \
0          Psychologist, counselling         0 -0.000278        34   
1  Special educational needs teacher         0 -0.000278        44   
2        Nature conservation officer         0 -0.000278        60   
3                    Patent attorney         0 -0.000278        55   
4     Dance movement psychotherapist         0 -0.000278        36   

   displacement  Time  
0     96.011245     0  
1     29.954105     0  
2    107.342986     0  
3    101.178169     0  
4     95.535506     0

In [31]:
data_test.head()

trans_date_trans_time            cc_num  \
0   2020-06-21 12:14:25  2291163933867244   
1   2020-06-21 12:14:33  3573030041201292   
2   2020-06-21 12:14:53  3598215285024754   
3   2020-06-21 12:15:15  3591919803438423   
4   2020-06-21 12:15:17  3526826139003047   

                               merchant        category    amt gender state  \
0                 fraud_Kirlin and Sons   personal_care   2.86      M    SC   
1                  fraud_Sporer-Keebler   personal_care  29.84      F    UT   
2  fraud_Swaniawski, Nitzsche and Welch  health_fitness  41.28      F    NY   
3                     fraud_Haley Group        misc_pos  60.05      M    FL   
4                 fraud_Johnston-Casper          travel   3.19      M    MI   

   city_pop                     job  is_fraud   recency  year_old  \
0    333497     Mechanical engineer         0 -0.000278        54   
1       302  Sales professional, IT         0 -0.000278        32   
2     34496       Librarian, public         0 -0.000278        52   
3     54767            Set designer         0 -0.000278        35   
4      1126      Furniture designer         0 -0.000278        67   

   displacement  Time  
0     29.260485    12  
1    109.084144    12  
2     75.126661    12  
3     27.608319    12  
4    123.069725    12

In [33]:
# we need to drop "merchant","job","trans_date_trans_time",'cc_num' columns because we have binned them, and also to avoid model complexity and multicollinearity,
# also we need to drop unique identifiers columns because those are for company's reference only and does not contribute in fraud prediction

# we are also removing state columnn, because it will create 49 extra dummy variables which will make our model computationally cost intensive model
# also in state column there are many states which have very less number of fraud transactions, so because of class imbalance those dummy variables can raise some unwanted cases
data.drop(["merchant","job","trans_date_trans_time",'cc_num','state'],axis=1,inplace=True)
data_test.drop(["merchant","job","trans_date_trans_time",'cc_num','state'],axis=1,inplace=True)


In [35]:
#spliting data into x_test,y_test,x_train,y_train
x_train=data.drop('is_fraud',axis=1)
y_train=data['is_fraud']
x_test=data.drop('is_fraud',axis=1)
y_test=data['is_fraud']

In [37]:
#dummy variable
x_train=pd.get_dummies(data=x_train,columns=['category','gender'],drop_first=True)
x_test=pd.get_dummies(data=x_test,columns=['category','gender'],drop_first=True)

In [38]:
x_train.head()

amt  city_pop   recency  year_old  displacement  Time  \
0    4.97      3495 -0.000278        34     96.011245     0   
1  107.23       149 -0.000278        44     29.954105     0   
2  220.11      4154 -0.000278        60    107.342986     0   
3   45.00      1939 -0.000278        55    101.178169     0   
4   41.96        99 -0.000278        36     95.535506     0   

   category_food_dining  category_gas_transport  category_grocery_net  \
0                     0                       0                     0   
1                     0                       0                     0   
2                     0                       0                     0   
3                     0                       1                     0   
4                     0                       0                     0   

   category_grocery_pos  category_health_fitness  category_home  \
0                     0                        0              0   
1                     1                        0              0   
2                     0                        0              0   
3                     0                        0              0   
4                     0                        0              0   

   category_kids_pets  category_misc_net  category_misc_pos  \
0                   0                  1                  0   
1                   0                  0                  0   
2                   0                  0                  0   
3                   0                  0                  0   
4                   0                  0                  1   

   category_personal_care  category_shopping_net  category_shopping_pos  \
0                       0                      0                      0   
1                       0                      0                      0   
2                       0                      0                      0   
3                       0                      0                      0   
4                       0                      0                      0   

   category_travel  gender_M  
0                0         0  
1                0         0  
2                0         1  
3                0         1  
4                0         1

In [39]:
x_test.head()

amt  city_pop   recency  year_old  displacement  Time  \
0    4.97      3495 -0.000278        34     96.011245     0   
1  107.23       149 -0.000278        44     29.954105     0   
2  220.11      4154 -0.000278        60    107.342986     0   
3   45.00      1939 -0.000278        55    101.178169     0   
4   41.96        99 -0.000278        36     95.535506     0   

   category_food_dining  category_gas_transport  category_grocery_net  \
0                     0                       0                     0   
1                     0                       0                     0   
2                     0                       0                     0   
3                     0                       1                     0   
4                     0                       0                     0   

   category_grocery_pos  category_health_fitness  category_home  \
0                     0                        0              0   
1                     1                        0              0   
2                     0                        0              0   
3                     0                        0              0   
4                     0                        0              0   

   category_kids_pets  category_misc_net  category_misc_pos  \
0                   0                  1                  0   
1                   0                  0                  0   
2                   0                  0                  0   
3                   0                  0                  0   
4                   0                  0                  1   

   category_personal_care  category_shopping_net  category_shopping_pos  \
0                       0                      0                      0   
1                       0                      0                      0   
2                       0                      0                      0   
3                       0                      0                      0   
4                       0                      0                      0   

   category_travel  gender_M  
0                0         0  
1                0         0  
2                0         1  
3                0         1  
4                0         1

In [46]:
#scaling the features
sc=StandardScaler()
x_train[['amt','city_pop','recency','year_old','displacement',"Time"]]=sc.fit_transform(x_train[['amt','city_pop','recency','year_old','displacement','Time']])

In [48]:
x_test[['amt','city_pop','recency','year_old','displacement',"Time"]]=sc.transform(x_test[['amt','city_pop','recency','year_old','displacement','Time']])

#training models to check which gives better results

In [51]:
#LogisticRegression model
model1=LogisticRegression(random_state=1).fit(x_train,y_train)
#KNeighborsClassifier model
model2=KNeighborsClassifier(n_neighbors=5).fit(x_train,y_train)
#Naive bayes classifier .... GaussionNB
model3=GaussianNB().fit(x_train,y_train)
#DecisionTreeClassifier
model4=DecisionTreeClassifier(random_state=1).fit(x_train,y_train)

In [52]:
#RandomForestClassifier 
model5=RandomForestClassifier(random_state=1).fit(x_train,y_train)

predicting using diffrnt models

In [53]:
#LogisticRegression
model1_pred=model1.predict(x_test)
#KNeighborsClassifier
model2_pred=model2.predict(x_test)
#naive bayes classifier ... GaussionNB
model3_pred=model3.predict(x_test)
#DecisionTreeclassifier
model4_pred=model4.predict(x_test)
#RandomForestclassifier
model5_pred=model5.predict(x_test)

In [59]:
result1=confusion_matrix(y_test,model1_pred)
result2=confusion_matrix(y_test,model2_pred)
result3=confusion_matrix(y_test,model3_pred)
result4=confusion_matrix(y_test,model4_pred)
result5=confusion_matrix(y_test,model5_pred)

In [72]:
allresult=[result1,result2,result3,result4,result5]
modelname=['LogisticRegression','Kneighborsclassifier','naive bayes classifier','DecisionTree','Randomforestclassifier']
accurecy_report={}
for i in range(len(allresult)):
    accurecy_report[modelname[i]]=(allresult[i][0][0]+allresult[i][1][1])/(allresult[i][0][0]+allresult[i][0][1]+allresult[i][1][0]+allresult[i][1][1])
    

In [73]:
accurecy_report

{'LogisticRegression': 0.007237742688028998,
 'Kneighborsclassifier': 0.9791497483949332,
 'naive bayes classifier': 0.0064094703761543945,
 'DecisionTree': 0.9387672315730619,
 'Randomforestclassifier': 0.948070256617888}

best accuracy we have got is 0.9791 by Kneighborsclassifier